In [29]:
import pandas as pd
import numpy as np
import os

In [34]:
folder='БД/Исходники/'
file=[i for i in os.walk(folder)][0][-1][-1]

zhzu=pd.read_csv(folder+file,dtype=str)

shifr=pd.read_excel('БД/классификаторы шифров.xlsx',dtype={'шифр': str})
zhzu['шифр']=zhzu['Номер договора'].astype(str).str[:3].astype(str)
zhzu=zhzu.merge(shifr,how='left',on='шифр')
zhzu=zhzu[zhzu['Линия бизнеса'].notna()]

# Даты для ЖЗУ
date_cols=['Дата заявления','Дата закрытия убытка','Дата отказа','Дата оплаты']
for col in date_cols:
    zhzu[col]=pd.to_datetime(zhzu[col])

zhzu['год заявления']=zhzu['Дата заявления'].dt.year
zhzu['месяц заявления']=zhzu['Дата заявления'].dt.month

int_cols=['Сумма заявленного убытка','РЗУ на дату заявления','РЗУ текущеее','Сумма оплаты']
for col in int_cols:
    zhzu[col]=zhzu[col].astype(float)

zhzu['Линия бизнеса_full']=np.where(zhzu['Линия бизнеса']!='02 - ОСАГО',
                                  zhzu['Линия бизнеса'],np.where(zhzu['ПВУ'].isna(),'02 - ОСАГО - Традиция',
                                                                '02 - ОСАГО'+' - '+zhzu['ПВУ']))

zhzu['Сумма оплаты']=np.where(zhzu['Линия бизнеса_full']!='03 - ДМС',
                              zhzu['Сумма оплаты'],zhzu['Сумма заявленного убытка'])

zhzu.to_csv('БД/Первичные данные/0-'+file[:-4]+'.csv',index=False)

#Обновление справочника
refbook=pd.read_excel('БД/Справочник первичных данных.xlsx',dtype=str)

refbook['Статус в работе']='0'

new_data=pd.DataFrame([['0-'+file[:-4],file[:-4][-2:]+'.'+file[:-4][-4:-2]+'.'+file[:-4][:2],'1','1']],
                      columns=refbook.columns)

refbook=pd.concat([refbook,new_data]).reset_index(drop=True)

if refbook.shape[0]<=3: pass
else:
    old_refbook=refbook[:-3]
    new_refbook=refbook[-3:]
    old_refbook['Наличие на сервере']='0'
    del_file=old_refbook[-1:].values[0][0]+'.csv'
    os.remove('БД/Первичные данные/'+del_file)
    refbook=pd.concat([old_refbook,new_refbook]).reset_index(drop=True)

refbook.to_excel('БД/Справочник первичных данных.xlsx',index=False)